In [1]:
#Imports
import gym
import tensorflow as tf
import numpy as np
import random

load_model           = False
environment_name     = "BipedalWalker-v2"
saver_file           = "./models/bipedal"
learning_rate        =.00005
frame_limit          = 500
discount_decay_rate  = .90
n_epochs             = 5001
random_action_probability_range  = [.0, .99]
random_action_repeat_steps_range = [ 1,  20]

    
# def helper_discount_rewards(rewards, discount_rate, begin_index, end_index):
#     #Takes in rewards and applies discount rate
#     discounted_rewards = np.zeros(len(rewards))
#     cumulative_rewards = 0
#     for step in reversed(range(len(rewards))):
#         cumulative_rewards = rewards[step] + cumulative_rewards * discount_rate
#         discounted_rewards[step] = cumulative_rewards
        
#     relevant_discounted_rewards = discounted_rewards[begin_index:end_index]
    
#     reward_mean = relevant_discounted_rewards.mean()
#     reward_std = relevant_discounted_rewards.std()
        
#     return [(discounted_reward - reward_mean)/reward_std for discounted_reward in discounted_rewards]

def helper_discount_rewards(rewards, discount_rate, begin_index, end_index):
    #Takes in rewards and applies discount rate
    discounted_rewards = []
    
    for i in range(len(rewards)):
        this_reward = 0.0
        exponent = 1
        for j in range(i,len(rewards)):
            this_reward += rewards[j]*discount_rate**exponent
            exponent += 1
        discounted_rewards.append(this_reward)
    return discounted_rewards
    
reward_test = [1,1,1,0,1,1,1]
print(helper_discount_rewards(reward_test, 0.9, 0, 0))



[4.0392279, 3.488031, 2.87559, 2.1951, 2.439, 1.71, 0.9]


In [2]:
# build the NN
reg_param = 0.00001

value_scale = 0.5  #how much does the value vs progressive gains influence the loss

activ = tf.nn.sigmoid

n_inputs = 4
n_outputs = 4
n_obs_inputs = 24

n_hidden1 = 256
n_hidden2 = 256
n_hidden3 = 256
n_hidden4 = 256
n_hidden5 = 256
n_hidden6 = 128
n_hidden7 = 64
n_hidden8 = 32
initializer = tf.contrib.layers.variance_scaling_initializer()

tf.reset_default_graph()

tf_input_obs = tf.placeholder(tf.float32, shape=(None, n_obs_inputs))
tf_input_action = tf.placeholder(tf.float32, shape=(None, n_inputs))
tf_input_learning_rate = tf.placeholder(tf.float32)
tf_input_reward = tf.placeholder(tf.float32)

input_layer = tf.layers.dense(tf_input_obs, n_obs_inputs, activation=activ, name="input", kernel_initializer=initializer)

hidden1 = tf.layers.dense(input_layer, n_hidden1, activation=activ, name="hidden1", kernel_initializer=initializer)
hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=activ, name="hidden2", kernel_initializer=initializer)
hidden3 = tf.layers.dense(hidden2, n_hidden3, activation=activ, name="hidden3", kernel_initializer=initializer)
hidden4 = tf.layers.dense(hidden3, n_hidden4, activation=activ, name="hidden4", kernel_initializer=initializer)
hidden5 = tf.layers.dense(hidden4, n_hidden5, activation=activ, name="hidden5", kernel_initializer=initializer)
hidden6 = tf.layers.dense(hidden5, n_hidden6, activation=activ, name="hidden6", kernel_initializer=initializer)
hidden7 = tf.layers.dense(hidden6, n_hidden7, activation=activ, name="hidden7", kernel_initializer=initializer)
hidden8 = tf.layers.dense(hidden7, n_hidden8, activation=activ, name="hidden8", kernel_initializer=initializer)

logits = tf.layers.dense(hidden8, n_outputs, name="output")
value  = tf.layers.dense(hidden8, 1,         name='value')

#pg_loss    = tf.reduce_mean(1.0 - tf_input_reward) * tf.reduce_mean(tf.square(logits-tf_input_action))
#value_loss = value_scale * tf.reduce_mean(tf.square(1.0- tf_input_reward))
#loss = pg_loss + value_loss

#loss = tf.reduce_mean(tf.square(logits-tf_input_action))
loss = tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=tf_input_action)
optimizer = tf.train.AdamOptimizer(tf_input_learning_rate)
training_op = optimizer.minimize(loss)

# regularization loss
#policy_network_variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="policy_network")
        
# compute policy loss and regularization loss
# cross_entropy_loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_input_action)
# pg_loss            = tf.reduce_mean(cross_entropy_loss)
# reg_loss           = tf.reduce_sum([tf.reduce_sum(tf.square(x)) for x in policy_network_variables])
# loss               = pg_loss + reg_param * reg_loss

# gradients = optimizer.compute_gradients(loss)

# compute policy gradients
# for i, (grad, var) in enumerate(gradients):
#     if grad is not None:
#         gradients[i] = (grad * tf_input_reward, var)


# training_op = optimizer.apply_gradients(gradients)

init = tf.global_variables_initializer()
saver = tf.train.Saver()


In [ ]:
env = gym.make(environment_name)
global_step = 0

with tf.Session() as sess:
    if load_model:
        print("Loading existing model before training: {}".format(saver_file))
        saver.restore(sess, saver_file)
    else:
        print("Creating new model before training: {}".format(saver_file))
        sess.run(init)
    
    for epoch in range(n_epochs):  #play n_epochs games
        game_step_counter = 0
        random_action_counter = 0
        
        all_actions = []
        all_rewards = []
        all_observations = []
        all_logits = []

        observation = env.reset()
        input_action = env.action_space.sample()  #set an initial input value
        
        while True:
            random_action_probability  = random.uniform(random_action_probability_range[0], random_action_probability_range[1])
            random_action_repeat_steps = random.randint(random_action_repeat_steps_range[0], random_action_repeat_steps_range[1])

            if epoch % 1 == 0:
                random_action_probability = 0.0
                
            #feed data into the AI to get action from the AI
            feed_dict = {tf_input_obs    : np.reshape(observation, (1, len(observation))), 
                         tf_input_action : np.reshape(input_action, (1, len(input_action))),
                         tf_input_learning_rate: 0.0,
                         tf_input_reward: 0.0}
            logits_out = sess.run([logits], feed_dict=feed_dict)

            # set the input value to feed into the next step
            
            if random_action_counter > 0 and random_action_counter < random_action_repeat_steps:
                random_action_counter += 1
            elif random_action_counter >= random_action_repeat_steps:
                random_action_counter = 0
            elif random.uniform(0.0, 1.0) < random_action_probability:
                input_action = env.action_space.sample()
                random_action_counter += 1
                #print("Random action {}: ".format(input_action))
            else:
                input_action = logits_out[0][0]

            #run the next step given the input from the logits
            observation, reward_float, done_bool, info_dict = env.step(input_action)
                
            #add the data to our lists
            all_observations.append(observation)
            all_logits.append(logits_out[0][0])
            all_actions.append(input_action)
            all_rewards.append(reward_float)

            if done_bool:
                break
                
            if game_step_counter > frame_limit:
                break

            if epoch % 1 == 0:
                env.render()  #display the current frame.
               
            game_step_counter += 1
        
        if epoch % 50 == 0 and epoch > 0:
            print("Saving model at epoch {}: {}".format(epoch, saver_file))
            saver.save(sess, saver_file)

        num_frames = len(all_actions)
        discounted_rewards = helper_discount_rewards(all_rewards, discount_decay_rate, 0, num_frames) #-1-frames_to_skip_end
        discounted_rewards_median = np.median(discounted_rewards)
        discounted_rewards_mean = np.mean(discounted_rewards)
        
        loss_out_sum = 0.0
        training_frame_counter = 0
        #use stored data to train
        for i in range(num_frames): 
            train_input_action = all_actions[i]
            
            if discounted_rewards[i] + all_rewards[i] >= 1.0:
                train_input_action = all_actions[i]
                training_frame_counter += 1
            else:
                continue
            
            feed_dict = {tf_input_obs : np.reshape(all_observations[i], (1, len(all_observations[i]))), 
                         tf_input_action : np.reshape(train_input_action, (1, len(train_input_action))),
                         tf_input_learning_rate: learning_rate,
                         tf_input_reward: discounted_rewards[i]}
        
            loss_out, _, logits_out = sess.run([loss, training_op, logits], feed_dict=feed_dict)
            loss_out_sum += abs(loss_out)

        if training_frame_counter > 0:
            print("Epoch: " + str(epoch) + ", reward total: " + str(np.sum(all_rewards)) + ", frames trained: " + str(100*training_frame_counter/num_frames) + "% , average loss: " + str(loss_out_sum/training_frame_counter))
        else:
            print("Epoch: " + str(epoch) + ", reward total: " + str(np.sum(all_rewards)) + ", frames trained: " + str(training_frame_counter/num_frames) + "%")
        #TODO - should learning rate decrease over time?
        print("")


WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
Creating new model before training: ./models/bipedal
Epoch: 0, reward total: -95.15154892678373, frames trained: 18.181818181818183% , average loss: [[0.22185704 0.4891042  0.4268044  0.7381619 ]]

Epoch: 1, reward total: -96.49784169271092, frames trained: 17.6% , average loss: [[0.03749263 0.37573993 0.29450443 0.7242451 ]]

Epoch: 2, reward total: -122.17897984290681, frames trained: 0.0%

Epoch: 3, reward total: -122.36671444379849, frames trained: 0.0%

Epoch: 4, reward total: -31.71634482024176, frames trained: 0.0%



In [ ]:
print(num_frames)
print(len(all_observations))
print(len(all_actions))
print(len(all_rewards))
print(len(all_logits))
print(len(discounted_rewards))

values = discounted_rewards

for value in all_rewards:
    print(value)


In [3]:

env = gym.make(environment_name)
observation = env.reset()

print("obs.shape: {}".format(observation.shape)) #obs.shape: (210, 160, 3)
print("env.action_space: {}".format(env.action_space)) #env.action_space: Discrete(9)


for _ in range(14):
    env.render()
    action = env.action_space.sample()
    print("action: {}".format(action))
    print(type(action))
    observation, reward_float, done_bool, info_dict = env.step(action) # take a random action
    
    print("observation: {}".format(observation))
    print("reward_float: {}".format(reward_float))
    print("done_bool: {}".format(done_bool))
    print("info_dict: {}".format(info_dict))

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
obs.shape: (24,)
env.action_space: Box(4,)
action: [0.09762701 0.43037874 0.20552675 0.08976637]
<class 'numpy.ndarray'>
observation: [ 0.00245932 -0.00691658  0.00590502  0.01961307 -0.29061297 -0.71013689
  1.4722293   0.99358988  1.          0.30079773 -0.01692069  0.1633203
  0.33325795  1.          0.45289648  0.45803979  0.4740701   0.50296849
  0.54874223  0.61897415  0.72858626  0.91021472  1.          1.        ]
reward_float: -0.01773284123155096
done_bool: False
info_dict: {}
action: [-0.1526904   0.29178822 -0.12482557  0.78354603]
<class 'numpy.ndarray'>
observation: [ 0.00255965  0.00256462  0.00806203  0.00679228 -0.01382717 -0.39574927
  1.08375649  0.          1.          0.26067981 -0.41715193  0.26995051
  0.24234966  1.          0.45600694  0.46118557  0.47732601  0.50642